# Задание 2

В датасете имеем файл с самими письмами, файл с расшифровкой PersonID, файл с Id получателей писем.

In [59]:
%matplotlib inline

import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from sklearn.cross_validation import train_test_split
from sklearn.datasets import make_classification
from functools import reduce
from copy import deepcopy
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.base import ClassifierMixin, BaseEstimator
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from stemming.porter2 import stem
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.cluster.hierarchical import AgglomerativeClustering
from sklearn.manifold import TSNE

In [2]:
data = pd.read_csv('Emails.csv')
print len(data)
data.head()

7945


,Id,DocNumber,MetadataSubject,MetadataTo,MetadataFrom,SenderPersonId,MetadataDateSent,MetadataDateReleased,MetadataPdfLink,MetadataCaseNumber,...,ExtractedTo,ExtractedFrom,ExtractedCc,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedDateReleased,ExtractedReleaseInPartOrFull,ExtractedBodyText,RawText
0,1,C05739545,WOW,H,"Sullivan, Jacob J",87.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739545...,F-2015-04841,...,NaN,"Sullivan, Jacob J <Sullivan11@state.gov>",NaN,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,05/13/2015,RELEASE IN FULL,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,2,C05739546,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,NaN,2011-03-03T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739546...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739546,05/13/2015,RELEASE IN PART,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,3,C05739547,CHRIS STEVENS,;H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739547...,F-2015-04841,...,B6,"Mills, Cheryl D <MillsCD@state.gov>","Abedin, Huma","Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,05/14/2015,RELEASE IN PART,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,4,C05739550,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",32.0,2012-09-12T04:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH2/DOC_0C05739550...,F-2015-04841,...,NaN,"Mills, Cheryl D <MillsCD@state.gov>","Mitchell, Andrew B","Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,05/13/2015,RELEASE IN PART,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,5,C05739554,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,80.0,2011-03-11T05:00:00+00:00,2015-05-22T04:00:00+00:00,DOCUMENTS/HRC_Email_1_296/HRCH1/DOC_0C05739554...,F-2015-04841,...,NaN,NaN,NaN,NaN,F-2015-04841,C05739554,05/13/2015,RELEASE IN PART,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


Посмотрим на неотображенные в head() колонки

In [3]:
print data.columns
print data.ExtractedSubject[:10]
print data.MetadataDocumentClass[:10]
print data.MetadataDocumentClass[5000:5010]

Index([u'Id', u'DocNumber', u'MetadataSubject', u'MetadataTo', u'MetadataFrom',
       u'SenderPersonId', u'MetadataDateSent', u'MetadataDateReleased',
       u'MetadataPdfLink', u'MetadataCaseNumber', u'MetadataDocumentClass',
       u'ExtractedSubject', u'ExtractedTo', u'ExtractedFrom', u'ExtractedCc',
       u'ExtractedDateSent', u'ExtractedCaseNumber', u'ExtractedDocNumber',
       u'ExtractedDateReleased', u'ExtractedReleaseInPartOrFull',
       u'ExtractedBodyText', u'RawText'],
      dtype='object')
0                                              FW: Wow
1                                                  NaN
2                                    Re: Chris Stevens
3                      FVV: Cairo Condemnation - Final
4                                                  NaN
5    Meet The Right Wing Extremist Behind Anti-Musl...
6    FW: Anti-Muslim film director in hiding, follo...
7                                                  NaN
8                             FVV: Secretary's r

ExtractedSubject и MetadataDocumentClass могут пригодиться. Оставим только некоторые колонки.

In [4]:
X_train = data.loc[:, ['MetadataSubject', 'MetadataTo', 'MetadataFrom', 'MetadataDocumentClass', 'SenderPersonId', 'ExtractedSubject', 
                       'ExtractedDateSent', 'ExtractedCaseNumber', 'ExtractedDocNumber', 'ExtractedBodyText', 
                       'RawText']]
X_train.head()

,MetadataSubject,MetadataTo,MetadataFrom,MetadataDocumentClass,SenderPersonId,ExtractedSubject,ExtractedDateSent,ExtractedCaseNumber,ExtractedDocNumber,ExtractedBodyText,RawText
0,WOW,H,"Sullivan, Jacob J",HRC_Email_296,87.0,FW: Wow,"Wednesday, September 12, 2012 10:16 AM",F-2015-04841,C05739545,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
1,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,H,NaN,HRC_Email_296,NaN,NaN,NaN,F-2015-04841,C05739546,"B6\nThursday, March 3, 2011 9:45 PM\nH: Latest...",UNCLASSIFIED\nU.S. Department of State\nCase N...
2,CHRIS STEVENS,;H,"Mills, Cheryl D",HRC_Email_296,32.0,Re: Chris Stevens,"Wednesday, September 12, 2012 11:52 AM",F-2015-04841,C05739547,Thx,UNCLASSIFIED\nU.S. Department of State\nCase N...
3,CAIRO CONDEMNATION - FINAL,H,"Mills, Cheryl D",HRC_Email_296,32.0,FVV: Cairo Condemnation - Final,"Wednesday, September 12,2012 12:44 PM",F-2015-04841,C05739550,NaN,UNCLASSIFIED\nU.S. Department of State\nCase N...
4,H: LATEST: HOW SYRIA IS AIDING QADDAFI AND MOR...,"Abedin, Huma",H,HRC_Email_296,80.0,NaN,NaN,F-2015-04841,C05739554,"H <hrod17@clintonemail.com>\nFriday, March 11,...",B6\nUNCLASSIFIED\nU.S. Department of State\nCa...


Вначале будем анализировать только текст письма (вся остальная содержательная информация у нас уже выделена). Посмотрим, хорошо ли выделен текст в датасете

In [5]:
print data.RawText[0]
print data.ExtractedBodyText[0]

UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.
RELEASE IN FULL
From: Sullivan, Jacob J <Sullivan11@state.gov>
Sent: Wednesday, September 12, 2012 10:16 AM
To:
Subject: FW: Wow
From: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi
Sent: Wednesday, September 12, 2012 10:09 AM
To: Sullivan, Jacob J
Subject: Wow
What a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me
too
UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247

nan


In [6]:
print data.RawText[2000]
print data.ExtractedBodyText[2000]

UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05762608 Date: 06/30/2015
RELEASE IN FULL
From:
Sent:
To:
Subject:
Mills, Cheryl D <MillsCD@state.gov>
Thursday, June n, 2009 5:42 AM
Fw: COS call 6/10/09
Fyi
From: Toiv, Nora F
To: Mills, Cheryl D; Sullivan, Jacob 3
Sent: Wed Jun 10 16:21:39 2009 •
Subject: COS call 6/10/09
Called out HRC as especially helpful in making calls to the Hill on passing the supplemental approps bill.
Nora Toiv
Office of the Secretary
202-647-8633
52
UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05762608 Date: 06/30/2015

Mills, Cheryl D <MillsCD@state.gov>
Thursday, June n, 2009 5:42 AM
Fw: COS call 6/10/09
Fyi


In [7]:
print data.RawText[7000]
print data.ExtractedBodyText[7000]

UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05773760 Date: 08/31/2015
RELEASE IN PART
B5
From:
Sent:
To:
Subject:
Sullivan, Jacob.) <SullivanJJ@state.gov>
Monday, December 27, 2010 7:46 AM
Khodorkovsky verdict
Embassy Moscow and Bill recommend that we put out the below statement in your name.
STATEMENT BY SECRETARY CLINTON
Statement on the Verdict in the Khodorkoysky/Lebedev Case
UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05773760 Date: 08/31/2015

Sullivan, Jacob.) <SullivanJJ@state.gov>
Monday, December 27, 2010 7:46 AM
Khodorkovsky verdict
Embassy Moscow and Bill recommend that we put out the below statement in your name.
STATEMENT BY SECRETARY CLINTON
Statement on the Verdict in the Khodorkoysky/Lebedev Case


Судя по всему, не очень. Вручную удалим ненужные строки, содержащие некоторую ненужную или уже выделенную информацию о письме.

In [8]:
X = X_train.RawText
bad_start = ['UNCLASSIFIED', 'RELEASE', 'From:', 'Sent:', 'To:', 'Subject:', 'Case', 'Doc', 'Date:', 'Fw:', 
            'Monday,', 'Tuesday,', 'Wednsday,', 'Thursday,', 'Friday,', 'Saturday,', 'Sunday,']

new_X = []
for i, text in enumerate(X):
    new_text = []
    for line in text.split('\n'):
        first_word = line.split(' ')[0]
        if not (first_word in bad_start):
            new_text.append(line)
    new_X.append('\n'.join(new_text))

In [9]:
print data.RawText[0]
print new_X[0]

UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER.
RELEASE IN FULL
From: Sullivan, Jacob J <Sullivan11@state.gov>
Sent: Wednesday, September 12, 2012 10:16 AM
To:
Subject: FW: Wow
From: Brose, Christian (Armed Services) (mailto:Christian_Brose@armed-servic,essenate.govi
Sent: Wednesday, September 12, 2012 10:09 AM
To: Sullivan, Jacob J
Subject: Wow
What a wonderful, strong and moving statement by your boss. please tell her how much Sen. McCain appreciated it. Me
too
UNCLASSIFIED
U.S. Department of State
Case No. F-2015-04841
Doc No. C05739545
Date: 05/13/2015
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIVE INFORMATION & REDACTIONS. NO FOIA WAIVER. STATE-5CB0045247

U.S. Department of State
STATE DEPT. - PRODUCED TO HOUSE SELECT BENGHAZI COMM.
SUBJECT TO AGREEMENT ON SENSITIV

In [10]:
print data.RawText[2000]
print new_X[2000]

UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05762608 Date: 06/30/2015
RELEASE IN FULL
From:
Sent:
To:
Subject:
Mills, Cheryl D <MillsCD@state.gov>
Thursday, June n, 2009 5:42 AM
Fw: COS call 6/10/09
Fyi
From: Toiv, Nora F
To: Mills, Cheryl D; Sullivan, Jacob 3
Sent: Wed Jun 10 16:21:39 2009 •
Subject: COS call 6/10/09
Called out HRC as especially helpful in making calls to the Hill on passing the supplemental approps bill.
Nora Toiv
Office of the Secretary
202-647-8633
52
UNCLASSIFIED U.S. Department of State Case No. F-2014-20439 Doc No. C05762608 Date: 06/30/2015

Mills, Cheryl D <MillsCD@state.gov>
Fyi
Called out HRC as especially helpful in making calls to the Hill on passing the supplemental approps bill.
Nora Toiv
Office of the Secretary
202-647-8633
52



Кажется, стало получше, не идеально, но вручную дальше сложно вырезать. Также есть ненужные небуквенные слова (телефоны, например), они отсеются после при векторизации, либо уберем потом вручную. В тексте не присутствуют никакие ненужные переносы. 

Oставим только буквенные слова и приведем все к lowercase. Удалим слова длины менее 3. 

Применим стемминг и сохраним результат отдельно.

In [11]:
raw_X = new_X
X = []
stem_X = []

for text in raw_X:
    a = re.findall(r'[A-z]{3,}', text)
    X.append(list(map(lambda x: x.lower(), a)))
    stem_X.append(list(map(lambda x: stem(x), X[-1])))

In [12]:
print X[0]
print stem_X[0]

['department', 'state', 'state', 'dept', 'produced', 'house', 'select', 'benghazi', 'comm', 'subject', 'agreement', 'sensitive', 'information', 'redactions', 'foia', 'waiver', 'what', 'wonderful', 'strong', 'and', 'moving', 'statement', 'your', 'boss', 'please', 'tell', 'her', 'how', 'much', 'sen', 'mccain', 'appreciated', 'too', 'department', 'state', 'state', 'dept', 'produced', 'house', 'select', 'benghazi', 'comm', 'subject', 'agreement', 'sensitive', 'information', 'redactions', 'foia', 'waiver', 'state']
['depart', 'state', 'state', 'dept', 'produc', 'hous', 'select', 'benghazi', 'comm', 'subject', 'agreement', 'sensit', 'inform', 'redact', 'foia', 'waiver', 'what', 'wonder', 'strong', 'and', 'move', 'statement', 'your', 'boss', 'pleas', 'tell', 'her', 'how', 'much', 'sen', 'mccain', 'appreci', 'too', 'depart', 'state', 'state', 'dept', 'produc', 'hous', 'select', 'benghazi', 'comm', 'subject', 'agreement', 'sensit', 'inform', 'redact', 'foia', 'waiver', 'state']


Считаем наиболее часто встречающиеся биграммы

In [13]:
import nltk
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(reduce(lambda x, y: x + y, X))
stem_finder = BigramCollocationFinder.from_words(reduce(lambda x, y: x + y, stem_X))
print '10 best bigramms for texts', finder.nbest(bigram_measures.raw_freq, 10) 
print '10 best bigramms for stemmed texts', stem_finder.nbest(bigram_measures.raw_freq, 10) 

10 best bigramms for texts [('original', 'message'), ('department', 'state'), ('unclassified', 'department'), ('and', 'the'), ('doc', 'date'), ('case', 'doc'), ('state', 'case'), ('for', 'the'), ('that', 'the'), ('with', 'the')]
10 best bigramms for stemmed texts [('origin', 'messag'), ('depart', 'state'), ('unclassifi', 'depart'), ('and', 'the'), ('doc', 'date'), ('case', 'doc'), ('state', 'case'), ('for', 'the'), ('that', 'the'), ('with', 'the')]


Теперь по pmi

In [14]:
print '10 best bigramms for texts', finder.nbest(bigram_measures.pmi, 10) 
print '10 best bigramms for stemmed texts', stem_finder.nbest(bigram_measures.pmi, 10) 

10 best bigramms for texts [('[antigone', 'collective]'), ('[drug', 'traffickers]'), ('[easy', 'women]'), ('[flotilla', 'passengers]'), ('[gonzalez', 'murder]'), ('[inter', 'contktenta[l'), ('[john', 'carland]'), ('[justice', 'department]'), ('[last', 'week]'), ('[mai', 'corect')]
10 best bigramms for stemmed texts [('[antigon', 'collective]'), ('[drug', 'traffickers]'), ('[easi', 'women]'), ('[flotilla', 'passengers]'), ('[gonzalez', 'murder]'), ('[inter', 'contktenta[l'), ('[john', 'carland]'), ('[justic', 'department]'), ('[last', 'week]'), ('[mai', 'corect')]


Видим, что по pmi находятся пары содержательных слов, в отличие от обычных вероятностей. 

Далее будем работать со "стеммизированными" данными.

In [32]:
X_train = list(map(lambda s: ' '.join(s), stem_X))

Векторизуем наши тексты, снизим размерность и применим кластеризацию с n_clusters = 30. 

In [33]:
vectorizer = TfidfVectorizer(max_df=1000, min_df=10)
matrix = vectorizer.fit_transform(X_train)
print 'tfidf'
svd = TruncatedSVD(n_components=200, random_state=123)
features = svd.fit_transform(matrix)
print 'svd'

kmeans = KMeans(n_clusters=30, random_state=1)
labels = kmeans.fit_predict(features)

tfidf
svd


In [34]:
print features[:10]

[[  3.42182060e-01   7.63395504e-01   8.20838349e-02 ...,  -1.97692878e-02
   -1.28428004e-02  -2.83902700e-03]
 [  3.45953918e-01   3.73795796e-01  -6.67410630e-02 ...,   3.51952115e-02
    3.82374799e-03   1.57180854e-02]
 [  3.31106771e-01   6.89930591e-01   1.54962678e-01 ...,  -2.36266740e-02
   -2.45425202e-03   9.91904629e-03]
 ..., 
 [  3.58950397e-01   4.00913984e-01  -5.50304859e-02 ...,   3.00606049e-02
    4.34159085e-03   1.26583588e-02]
 [  4.17375814e-01   4.30965260e-01  -6.55119646e-02 ...,   1.93082280e-04
    4.28286867e-02  -1.56693925e-02]
 [  4.13191688e-01   6.22976549e-01  -3.99233835e-02 ...,  -8.96533954e-03
   -2.91271105e-02  -8.93404346e-03]]


Важные слова выберем с помощью tfidf. Объединим все тексты с одной меткой в один, для получившегося набора из 30 документов применим tfidf с выделением только нечасто встречающихся слов.

In [95]:
def show_clusters(n_clusters, labels):
    themes = []

    for label in range(n_clusters):
        themes.append(' '.join(list(map(lambda s: ' '.join(s), np.array(stem_X)[np.array(labels) == label]))))

    vectorizer = TfidfVectorizer(max_df=0.2)
    res = np.array(vectorizer.fit_transform(themes).toarray())

    for label in range(n_clusters):
        indices = np.array(list(reversed(np.argsort(res[label]).tolist())))
        print label, np.array(vectorizer.get_feature_names())[indices[:10]], '\n'

In [96]:
show_clusters(30, labels)

0 [u'vladivostock' u'harlan' u'siddharth' u'dalton' u'mohanda' u'reynold'
 u'slim' u'dellum' u'farnsworth' u'greenland'] 

1 [u'cfr' u'rosal' u'lamplight' u'madeeha' u'manuel' u'jock' u'ashrafm'
 u'baer' u'input' u'quak'] 

2 [u'dup' u'unionist' u'sinn' u'fein' u'uup' u'devolv' u'allist' u'stormont'
 u'ulster' u'hillsborough'] 

3 [u'unscr' u'zzq' u'geolog' u'gentlest' u'gentlewoman' u'genton' u'gentontr'
 u'genyen' u'geo' u'geoff'] 

4 [u'assk' u'johna' u'mcmullen' u'pirro' u'velia' u'pdt' u'bog' u'moriarti'
 u'trr' u'ohtagaki'] 

5 [u'benghazi' u'foia' u'redact' u'waiver' u'scb' u'pouch' u'holder'
 u'somali' u'gloria' u'tripoli'] 

6 [u'fari' u'firri' u'moroccan' u'kouchen' u'standoff' u'iand' u'hongju'
 u'passport' u'belliard' u'edwidg'] 

7 [u'pdpa' u'patriarch' u'gore' u'diseas' u'pao' u'ecumen' u'chronic'
 u'hispan' u'bangladesh' u'innov'] 

8 [u'hasina' u'managua' u'grameen' u'berlusconi' u'rio' u'yunus' u'bca'
 u'tehran' u'kubisk' u'orna'] 

9 [u'pverve' u'gosl' u'surgic' u'wyn

Подберем число кластеров с помощью ситуэтного скора.

In [57]:
vectorizer = TfidfVectorizer(max_df=1000, min_df=10)
matrix = vectorizer.fit_transform(X_train)
scores = []

for n_clusters in [k * 5 for k in range(1, 10)]:
    kmeans = KMeans(n_clusters=n_clusters, random_state=1)
    scores.append(silhouette_score(matrix, kmeans.fit_predict(matrix)))
    print n_clusters

5
10
15
20
25
30
35
40
45


In [58]:
scores

[0.018560201493359078,
 0.023701118401724213,
 0.026134969331769387,
 -0.0054641984678143662,
 0.028807740628550838,
 -0.0059124561662981402,
 0.013088541156589313,
 -0.00082248425400642392,
 0.0091996293575547609]

Будем делить на 25 кластеров. Используем более аккуратный метод снижения размерности - добавим tSNE и попробуем применить Agglomerative Clustering. Можно использовать некоторые другие признаки - id отправителя и получателя, выделенную тему письма и тд, но сейчас делать этого не будем, ибо фокусируемся на кластеризации текстов.

In [65]:
vectorizer = TfidfVectorizer(max_df=1000, min_df=10)
matrix = vectorizer.fit_transform(X_train)
print 'tfidf'
svd = TruncatedSVD(n_components=20, random_state=123)
features_svd = svd.fit_transform(matrix)
print 'svd'
tsne = TSNE(n_components=5)
features = tsne.fit_transform(features_svd)
print 'tsne'

aclust = AgglomerativeClustering(n_clusters=25, affinity='cosine', linkage='complete')
new_labels = kmeans.fit_predict(features)

tfidf
svd
tsne


In [97]:
show_clusters(25, new_labels)

0 [u'brandenburg' u'oprah' u'moy' u'legrand' u'beller' u'restavek' u'governo'
 u'ushahidi' u'estado' u'jillian'] 

1 [u'tapper' u'morsi' u'transpir' u'granger' u'expeditionari' u'giuliani'
 u'outmatch' u'politic' u'iftar' u'tunisia'] 

2 [u'samoa' u'faleomavaega' u'bahtiyar' u'maquiladora' u'samoan' u'airlift'
 u'wike' u'arkin' u'rascon' u'paso'] 

3 [u'amazon' u'utf' u'node' u'mailcontrol' u'ecx' u'phrma' u'altohiway'
 u'kindl' u'tauzin' u'sheikha'] 

4 [u'sejm' u'pani' u'poset' u'preze' u'andrzej' u'stanu' u'vmi' u'aboud'
 u'hazza' u'dowodca'] 

5 [u'timesonlin' u'murdoch' u'mandelson' u'lellouch' u'populus' u'tol'
 u'apologis' u'sinn' u'castrat' u'manifesto'] 

6 [u'surgic' u'gillibrand' u'aba' u'pdt' u'pih' u'fester' u'aloha' u'surgeon'
 u'amano' u'bog'] 

7 [u'patriarch' u'ecumen' u'orthodox' u'ergenekon' u'istanbul' u'archon'
 u'jamal' u'bartholomew' u'constantinopl' u'asp'] 

8 [u'fistula' u'goma' u'lyn' u'kivu' u'congoles' u'lamego' u'congo' u'lusi'
 u'tost' u'maniema'] 

9 [u'

In [98]:
kmeans = KMeans(n_clusters=25, random_state=1)
new_klabels = kmeans.fit_predict(features)
show_clusters(25, new_klabels)

0 [u'vol' u'hrd' u'hvn' u'wht' u'tot' u'mod' u'postconflict' u'blk' u'hasina'
 u'wldn'] 

1 [u'clntinet' u'evit' u'fe' u'herebi' u'diagnost' u'vanc' u'mswashdcib'
 u'sis' u'geller' u'bmena'] 

2 [u'windrush' u'obl' u'llosa' u'broadway' u'varga' u'others' u'griot'
 u'strikethrough' u'bi' u'messagelab'] 

3 [u'udal' u'filibust' u'pou' u'mccaskil' u'warner' u'bun' u'newyorkercom'
 u'kaidanow' u'freshman' u'gouvenman'] 

4 [u'managua' u'bca' u'kubisk' u'orna' u'loh' u'arola' u'posivak' u'drucker'
 u'milton' u'handov'] 

5 [u'carleto' u'loach' u'brandenburg' u'oak' u'moy' u'forman' u'edinburgh'
 u'domenici' u'restavek' u'hubbard'] 

6 [u'morsi' u'transpir' u'tapper' u'fisa' u'indyk' u'iftar' u'granger'
 u'expeditionari' u'rushdi' u'gadhafi'] 

7 [u'hasina' u'fari' u'firri' u'maquiladora' u'kouchen' u'pml' u'bajnai'
 u'iand' u'rascon' u'standoff'] 

8 [u'lewinski' u'democracyinact' u'capweb' u'reno' u'vol' u'moynihan'
 u'codex' u'tobacco' u'jsp' u'dis'] 

9 [u'stiglitz' u'skousen' u'osborn' 

Асессорам можно предоставить эти данные, показать несколько соответствующим кластерам писем (10 случайных, например), и попросить определить, действительно ли они принадлежат одной теме. Если да, то попросить назвать эту тему, а также для каждого слова из 10 выделенных сказать, подходит ли оно к этой теме. Далее, можно из полученных 11 ответов (0 и 1) посчитать что-нибудь. Например, скор кластеризации можно положить равным <сумма по всем кластерам>0.5 * I['тексты принадлежат одной тематике'] + 0.05 * (<сумма по всем наиболее важным терминам>I['термин подходит к этой теме']). В данном случае большая проблема состоит в интерпретации тем и даже переводе слов - для этого нужно разбираться в политике и отлично владеть достаточно специфичным английским языком (с кучей имен собственных)

In [93]:
def write_test(filename, label):
    with open(filename, 'w') as f:
        themes = []

        for label in range(25):
            themes.append(' '.join(list(map(lambda s: ' '.join(s), np.array(stem_X)[np.array(labels) == label]))))

        vectorizer = TfidfVectorizer(max_df=5)
        res = np.array(vectorizer.fit_transform(themes).toarray())
        for label in range(25):
            f.write('\n-----\nLABEL ' + str(label) + '\n')
            for text in np.array(data.RawText)[np.array(labels) == label][:10]:
                f.write(text)
                f.write('\n--\n')
                
            f.write('-----\nWORDS\n-----')
            indices = np.array(list(reversed(np.argsort(res[label]).tolist())))
            for i in range(10):
                f.write((vectorizer.get_feature_names())[indices[i]])
                f.write('\t')

In [99]:
write_test('test1.txt', new_labels)
write_test('test2.txt', new_klabels)